In [31]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
import numpy

In [5]:
df = pd.read_csv('ner_dataset_dota2_chat.csv', encoding = "utf_8", sep=';')

In [6]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,yes,RB,O
1,NaN,dog,NN,B-bad
2,NaN,yeah,JJ,O
3,NaN,fast,JJ,O
4,NaN,and,CC,O
5,NaN,furious,JJ,O
6,NaN,too,RB,O
7,NaN,fas,JJ,O
8,NaN,haha,NN,O
9,NaN,sad,JJ,O


In [7]:
df.describe()

,Sentence #,Word,POS,Tag
count,50,636,636,636
unique,50,316,29,8
top,Sentence: 25,you,NN,O
freq,1,26,193,523


#### Observations : 
* There are total 47959 sentences in the dataset.
* Number unique words in the dataset are 35178.
* Total 17 lables (Tags).

In [8]:
df['Tag'].unique()

array(['O', 'B-bad', 'I-pra', 'B-pra', 'I-bad', 'B-her', 'B-hero',
       'I-hero'], dtype=object)

In [9]:
df.isnull().sum()

Sentence #    586
Word            0
POS             0
Tag             0
dtype: int64

In [10]:
df = df.fillna(method = 'ffill')

In [11]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [12]:
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'yes dog yeah fast and furious too fas haha sad'

In [13]:
sent = getter.get_text()
print(sent)

[('yes', 'RB', 'O'), ('dog', 'NN', 'B-bad'), ('yeah', 'JJ', 'O'), ('fast', 'JJ', 'O'), ('and', 'CC', 'O'), ('furious', 'JJ', 'O'), ('too', 'RB', 'O'), ('fas', 'JJ', 'O'), ('haha', 'NN', 'O'), ('sad', 'JJ', 'O')]


Getting all the sentences in the dataset.

In [14]:
sentences = getter.sentences

In [15]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [16]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [19]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [20]:
y_pred = crf.predict(X_test)

In [21]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.8958272964358158


In [22]:
report = flat_classification_report(y_test, y_pred)
print(report)

precision    recall  f1-score   support

       B-bad       1.00      0.40      0.57         5
       B-her       0.00      0.00      0.00         4
       B-pra       1.00      1.00      1.00         4
       I-bad       1.00      0.50      0.67         2
       I-pra       1.00      1.00      1.00         4
           O       0.91      1.00      0.95        83

    accuracy                           0.92       102
   macro avg       0.82      0.65      0.70       102
weighted avg       0.89      0.92      0.90       102

